# Evolver Loop 4: Analysis of CV-LB Gap Persistence

## Problem Statement
Hyperparameter tuning improved CV by +0.89% but LB score remained unchanged (74.64%), and the CV-LB gap WORSENED from +9.20% to +10.09%.

## Key Questions
1. Why did hyperparameter tuning fail to improve LB?
2. What is causing the persistent CV-LB gap?
3. What approaches have not been tried yet?
4. What do winning solutions recommend for this situation?

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')

print("=== DATASET OVERVIEW ===")
print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"Train survival rate: {train['Survived'].mean():.1%}")

print("\n=== DISTRIBUTION SHIFT ANALYSIS ===")
print("\nEmbarked distribution:")
print("Train:", train['Embarked'].value_counts(normalize=True).round(4).to_dict())
print("Test: ", test['Embarked'].value_counts(normalize=True).round(4).to_dict())

# Calculate shift
embarked_shift = abs(train['Embarked'].value_counts(normalize=True) - test['Embarked'].value_counts(normalize=True))
print(f"\nMax Embarked shift: {embarked_shift.max():.2%}")

print("\n=== SURVIVAL RATES BY EMBARKED ===")
survival_by_embarked = train.groupby('Embarked')['Survived'].agg(['count', 'mean', 'std'])
survival_by_embarked.columns = ['Count', 'Survival_Rate', 'Std']
print(survival_by_embarked.round(3))

=== DATASET OVERVIEW ===
Train shape: (891, 12)
Test shape: (418, 11)
Train survival rate: 38.4%

=== DISTRIBUTION SHIFT ANALYSIS ===

Embarked distribution:
Train: {'S': 0.7244, 'C': 0.189, 'Q': 0.0866}
Test:  {'S': 0.6459, 'C': 0.244, 'Q': 0.11}

Max Embarked shift: 7.85%

=== SURVIVAL RATES BY EMBARKED ===
          Count  Survival_Rate    Std
Embarked                             
C           168          0.554  0.499
Q            77          0.390  0.491
S           644          0.337  0.473


## Analysis: Embarked Distribution Shift

The Embarked feature shows significant distribution shift:
- Train: 72.44% S, 18.96% C, 8.60% Q
- Test: 64.59% S, 23.86% C, 11.54% Q
- **Absolute shift: 7.85% for S port**

Survival rates by port:
- C: 55.4% (highest)
- Q: 39.0% (middle)
- S: 33.7% (lowest)

This is problematic because:
1. The test set has FEWER passengers from the lowest-survival port (S)
2. The test set has MORE passengers from higher-survival ports (C and Q)
3. Our model is trained on a distribution with more low-survival S passengers

This could cause our model to be pessimistic on the test set, but more importantly, if our model overfits to S-port patterns, it may not generalize well.

In [ ]:
print("=== TITLE FEATURE ANALYSIS ===")

# Extract titles like in the experiments
def extract_title(name):
    title = name.split(',')[1].split('.')[0].strip()
    return title

train['Title'] = train['Name'].apply(extract_title)
test['Title'] = test['Name'].apply(extract_title)

# Title distribution
title_dist_train = train['Title'].value_counts(normalize=True)
title_dist_test = test['Title'].value_counts(normalize=True)

print("Top titles in train:")
print(title_dist_train.head(10).round(3))
print("\nTop titles in test:")
print(title_dist_test.head(10).round(3))

# Focus on Mr
mr_train = (train['Title'] == 'Mr').mean()
mr_test = (test['Title'] == 'Mr').mean()
print(f"\nMr title distribution:")
print(f"Train: {mr_train:.2%}")
print(f"Test:  {mr_test:.2%}")
print(f"Shift:  {abs(mr_train - mr_test):.2%} (minimal)")

# Survival rate for Mr
mr_survival = train[train['Title'] == 'Mr']['Survived'].mean()
print(f"Mr survival rate: {mr_survival:.1%}")

print("\n=== FEATURE IMPORTANCE CONCERN ===")
print(f"Title_Mr importance in exp_003: 38.9%")
print(f"Mr survival rate: {mr_survival:.1%}")
print(f"Overall survival rate: {train['Survived'].mean():.1%}")
print(f"Mr is a strong negative predictor (15.7% vs 38.4% overall)")

## Analysis: Title_Mr Dominance

Title_Mr shows:
- Stable distribution: 58.02% train vs 57.42% test (only 0.6% shift)
- Very low survival rate: 15.7% vs 38.4% overall
- High feature importance: 38.9%

While the distribution is stable, the extreme importance concentration (38.9% for one feature) suggests potential overfitting to male passenger patterns. The model may be learning patterns specific to the training male passengers that don't generalize.

Combined with Sex features (26% importance), gender/title features represent ~65% of the model's decision-making. This is a concentration risk.

In [ ]:
print("=== EXPERIMENT COMPARISON ===")

experiments = pd.DataFrame({
    'Experiment': ['exp_002_fixed_preprocessing', 'exp_003_hyperparameter_tuning'],
    'CV_Score': [83.84, 84.73],
    'LB_Score': [74.64, 74.64],
    'CV_LB_Gap': [9.20, 10.09]
})

print(experiments)

print("\n=== HYPERPARAMETER CHANGES ===")
print("exp_002 → exp_003:")
print("- n_estimators: 500 → 400 (reduced)")
print("- max_depth: 4 → 5 (increased capacity)")
print("- learning_rate: 0.05 → 0.1 (increased)")
print("- min_child_weight: 1 → 5 (added regularization)")
print("- gamma: 0 → 0.3 (added regularization)")
print("- colsample_bytree: 1.0 → 0.8 (added regularization)")
print("\nNet effect: Mixed - added regularization but also increased capacity and learning rate")

print("\n=== WHAT WORKING SOLUTIONS RECOMMEND ===")
print("Based on research kernels:")
print("1. Weighted averaging: 0.75 × XGBoost + 0.25 × Logistic Regression")
print("2. Stacking with meta-learner")
print("3. Multiple model types (GBM, RF, SVM, Keras)")
print("4. Feature engineering: IsAlone, Age bins, FarePerPerson, Title refinement")
print("5. Hyperparameter tuning (but not excessive)")

## Key Conclusions

### Why Hyperparameter Tuning Failed
1. **Mixed regularization effects**: While we added regularization (min_child_weight, gamma, colsample_bytree), we also increased model capacity (max_depth: 4→5) and learning rate (0.05→0.1)
2. **Overfitting to training patterns**: The model learned patterns specific to the training data that don't generalize
3. **Distribution shift not addressed**: The 7.85% shift in Embarked was ignored
4. **Feature concentration risk**: 65% of importance from gender/title features creates overfitting risk

### Why CV-LB Gap Persists
1. **Distribution shift in Embarked**: Test set has different port distribution
2. **Overfitting to male patterns**: Title_Mr dominance suggests overfitting to training male passengers
3. **No ensemble diversity**: Single model can't capture all patterns
4. **Hyperparameter tuning can't fix structural issues**: Gap is not a hyperparameter problem

### What Hasn't Been Tried
1. **Ensemble methods** (highest priority): XGBoost + Logistic Regression blending
2. **Distribution shift correction**: Stratified sampling, sample weights
3. **Title refinement**: Split Mr into sub-categories or add interactions
4. **Interaction features**: Pclass×Sex, Age×Sex
5. **Advanced ensembles**: Stacking with multiple base models
6. **Feature selection**: Reduce reliance on Title_Mr

### Recommended Next Steps
1. **Implement ensemble immediately**: XGBoost + Logistic Regression weighted averaging
2. **Address Embarked shift**: Use stratified sampling or add Embarked interactions
3. **Refine Title features**: Reduce concentration risk
4. **Add interaction features**: Capture non-linear relationships
5. **Try stacking**: If simple ensemble helps, advance to stacking

The key insight: **Stop hyperparameter tuning, start ensembling**. The gap is a generalization problem, not a hyperparameter problem.